In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

dataset = pd.read_csv("Coursera.csv")

# Function for course recommendation
def recommend_courses_for_job(job_title, user_skills):
    relevant_skills = ' '.join(job_title)
    user_skills += ' ' + relevant_skills

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataset["Skills"])
    user_tfidf = tfidf_vectorizer.transform([user_skills])

    similarities = cosine_similarity(user_tfidf, tfidf_matrix)

    matching_courses = []

    for index, similarity in enumerate(similarities[0]):
        if similarity > 0.2:
            matching_courses.append((dataset.loc[index, "Course Name"],
                                     dataset.loc[index, "University"],
                                     dataset.loc[index, "Course URL"],
                                     similarity))

    matching_courses.sort(key=lambda x: x[3], reverse=True)

    if matching_courses:
        print("Matching courses:")
        for course_name, university, course_url, similarity in matching_courses:
            print(f"Course Name: {course_name}")
            print(f"University: {university}")
            print(f"Course URL: {course_url}")
            print(f"Cosine Similarity: {similarity:.4f}")
            print("=" * 30)
    else:
        print("No matching courses found.")





# Function for job recommendation
def perform_job_search(interests, location, field_of_study, skills):
    try:
        # Construct the URL with the provided parameters
        url = f"https://www.linkedin.com/jobs/search/?keywords={interests}+{field_of_study}&location={location}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        job_listings = soup.find_all('div', class_='base-search-card__info')

        # Extract job titles and locations from the job listings
        job_data = []
        for job in job_listings:
            title_element = job.find('h3', class_='base-search-card__title')
            location_element = job.find('span', class_='job-search-card__location')

            # Check if any of the elements are None
            if title_element is None or location_element is None:
                continue

            # Get the text from the elements
            title = title_element.text.strip() if title_element.text else ""
            location = location_element.text.strip() if location_element.text else ""

            # Ensure both title and location are available before adding to job_data
            if title and location:
                job_data.append((title, location))

        return job_data

    except Exception as e:
        print("Error:", e)
        return []

# Main program
def main():
    print("Welcome to the recommendation system!")
    interests = input("Enter your job interests: ")
    location = input("Where are you located? ")
    field_of_study = input("What is your field of study? ")
    skills = input("Please enter your skills (comma-separated): ")

    recommended_jobs = perform_job_search(interests, location, field_of_study, skills)
    job_titles = [job[0] for job in recommended_jobs]

    if recommended_jobs:
        print("Recommended jobs:")
        for idx, (title, location) in enumerate(recommended_jobs, 1):
            print(f"{idx}. {title}\n   Location: {location}\n")
    else:
        print("No jobs found based on your input.")
    job_choice = int(input("Enter the number of the job you are interested in: ")) - 1

    if 0 <= job_choice < len(recommended_jobs):
        selected_job = recommended_jobs[job_choice]

        recommended_courses = recommend_courses_for_job(selected_job, skills)

        if recommended_courses:
            print(f"Recommended courses for {selected_job}:")
            for course in recommended_courses:
                print(course[0], " - University:", course[1], " - URL:", course[2], " - Similarity:", course[3])
        else:
            print("No matching courses found for the selected job.")
    else:
        print("Invalid job choice.")
if __name__ == "__main__":
    main()


Welcome to the recommendation system!
Enter your job interests: DAta
Where are you located? London
What is your field of study? CS
Please enter your skills (comma-separated): Python
Recommended jobs:
1. Product Manager
   Location: London, England, United Kingdom

2. Customer Success Executive -  🚀 Scale up - Real Time Data (SaaS)
   Location: London, England, United Kingdom

3. Customer Operations Manager
   Location: London, England, United Kingdom

4. Account Manager (UK/EMEA)
   Location: London, England, United Kingdom

5. Freelance Business Process Analyst - 6M FTC
   Location: London, England, United Kingdom

6. Customer Success Associate
   Location: London, England, United Kingdom

7. Bioinformatician
   Location: Colindale, England, United Kingdom

8. Marketing Executive (JR100009)
   Location: London, England, United Kingdom

9. Label Coordinator - Warner Records/FFRR Records
   Location: London, England, United Kingdom

10. Online Customer Service Lead
   Location: London, 